In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../pax"))

from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import waveform_utils

from pax import core

pp = pprint.PrettyPrinter(depth=6)



In [124]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    event  = None
    df_zip_merged = pd.DataFrame()
    
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
        print("File: '" + str(zipfilename) + "'")
        print("   event_number: " + str(iPklFile))
        #clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)

      
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue

            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
                
        ############################################################################################
        ############################################################################################
        
        left                        = event.main_s2.left
        right                       = event.main_s2.right
        jsonfilename                = os.path.dirname(zipfilename) + '/pax_info.json'
        cfg                         = event_utils.getConfig(jsonfilename)
        df_channels_waveforms_top   = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        arr_summed_waveform_top_evt = waveform_utils.getSummedWaveformFromEvent(event, 'tpc_top')[left:right]
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top, event)[left:right]
        
        assert(waveform_utils.compareWaveforms(arr_summed_waveform_top_evt, arr_summed_waveform_top_df))
        
        

     
        ############################################################################################
        # Check the Summed Waveform S2 Integrals
        ############################################################################################
            
        sum_evt = np.sum(arr_summed_waveform_top_evt)
        sum_df  = np.sum(arr_summed_waveform_top_df).astype(np.float32)
        
        assert(sum_evt == sum_df)
        
        #sum_evt = round(sum_evt, 2)
        #sum_df  = round(sum_df , 2)
        #print("sum_evt: " + str(sum_evt))
        #print("sum_df:  " + str(sum_df ))
        
        
        
        ############################################################################################
        # Check the Channel Waveform S2 Integrals
        ############################################################################################
            
        print()
        
        ser = df_pkl_s2s.iloc[0]
        ser = ser[1:]
        ser = ser.as_matrix().astype(np.float32)
        ser = ser[np.nonzero(ser)]
        
        #display(ser)
        
        arr_sum_evt = ser.astype(np.float32)
        arr_sum_df  = df_channels_waveforms_top[:]['sum'].as_matrix().astype(np.float32)
        
        print()
        
        print(arr_sum_df.shape)
        print(arr_sum_evt.shape)
        
        print(arr_sum_df)
        print(arr_sum_evt)

        print(np.sum(sum_evt))
        print(np.sum(arr_sum_df))
        print(np.sum(arr_sum_evt))
        
        #assert(arr_sum_evt == arr_sum_df)
        
        
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged

    

In [125]:
import glob

####################################################################################################
####################################################################################################

#ver = 'test'
ver = datetime.datetime.now().strftime("%y%m%d%H%M")
ver = 'test'

nEventsPerFile = 2 #1000
nFilesZip      = 1 #200
nEvents        = nEventsPerFile * nFilesZip
dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
file_pkl     = 'merged/merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
lst_contents = glob.glob(dir_format)

if (nFilesZip == -1):
    
    nFilesZip    = len(lst_contents)

print()
print("PAX input files: " + str(nFilesZip))
print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged = processPklEvents(zipfilename)
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

print(file_pkl)

df.to_pickle(file_pkl)




PAX input files: 1

File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
File: '../pax_run/pax_output/Mar25/instructions_000000/XENON1T-0-000000000-000000999-000001000.zip'
   event_number: 0


(63,)
(52,)
[  1.43560374   2.19027233   3.73745537   1.96978021   6.69082451
   6.68689919   2.35583258   0.81786031   0.           3.10084391
   0.95770657   3.63397264   1.50121272   1.58797121   2.72301507
   1.29693341   4.22049713  14.6941185   12.98080826  11.53015804
   1.39322805   1.23892021   3.53061342   0.58457142   2.22669411
   2.66378117   1.40889943   0.86907828   0.95646775   1.27004671
   1.23100913   2.38115168  41.66978836  37.91341019   6.10058451
   4.79123116   1.86737633   1.14328623   1.12325406   0.78207356
   2.99171734   0.36640126  11.86457539  34.64745331   1.19479978
   2.96649671   1.29959655   1.47352922   1.0165422    3.24604702
   1.34756982  10.07618713   1.00133324   9.11426926   0.48168516
   1.30838346   1.58